In [42]:
import numpy as np
import scipy.stats as si
import pandas as pd

In [ ]:
def black_scholes(S, K, T, r, sigma, option_type="call", qtd_opcoes=1):
    """
    Calcula o prêmio da opção, Delta e a quantidade de ações necessárias para um Delta Hedge.

    Parâmetros:
    S : float - Preço atual do ativo subjacente
    K : float - Preço de exercício (strike)
    T : float - Tempo até o vencimento (em anos)
    r : float - Taxa de juros livre de risco (anual)
    sigma : float - Volatilidade do ativo (anual)
    option_type : str - Tipo da opção: "call" ou "put"
    qtd_opcoes : int - Quantidade de contratos de opções (cada contrato representa 100 opções)

    Retorna:
    tuple - (preço da opção, Delta, quantidade de ações para hedge)
    """

    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        preco = S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
        delta = si.norm.cdf(d1)
    elif option_type == "put":
        preco = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * si.norm.cdf(-d1)
        delta = si.norm.cdf(d1) - 1
    else:
        raise ValueError("O tipo da opção deve ser 'call' ou 'put'")

    # Quantidade de ações para hedge (Delta Hedge)
    qtd_acoes = delta * qtd_opcoes  # Considerando contratos de 100 opções cada

    return preco, delta, qtd_acoes

K = 100         # Strike da opção
r = 0.05        # Taxa de juros livre de risco (5% ao ano)
sigma = 0.2     # Volatilidade anual (20%)
qtd_opcoes = 1000  # Número de contratos de opções 

#Valores simulados
S = [100, 93.4, 88.7, 91.2, 84.9, 107.5, 116.3, 113.4, 106.7, 97.9, 
     119.6, 103.3, 109.8, 115.1, 118.2, 91.6, 89.4, 102.7, 108.1, 117.5, 
     95.2, 83.3, 110.6, 98.9, 107.2, 120.0, 113.7, 85.4, 119.4, 110.8, 
     104.5, 95.3, 96.8, 114.9, 97.2, 115.5, 108.4, 90.1, 121.0, 102.0, 
     119.8, 85.7, 104.8, 99.1, 111.3, 105,90, 85,80,200]

T  = [i / 52 for i in range(1, 51)]

base = pd.DataFrame(data={'Preco_Ativo':S,'Semana':T})
deltas = []
delta_anterior = 0
diffs_deltas = []
mov_acoes = []

for price,time in zip(S,T):
    # Calculando para uma CALL
    preco_call, delta_call, hedge_call = black_scholes(price, K, time, r, sigma, "call", qtd_opcoes)
    deltas.append(delta_call)

    #Fazendo a diff entre deltas para ajudste do Hedge
    diff_delta = delta_call - delta_anterior
    diffs_deltas.append(diff_delta)

    #Segurando o delta para a próxima diff
    delta_anterior = delta_call

#Colocando os deltas
base['Delta'] = deltas
#Vendo a diferença entre deltas
base['Diff'] = diffs_deltas
# Multiplicando pelo n de contratos para saber qual movimentação deve ser feita para manter o delta neutral
base["Mov_Acoes"] = qtd_opcoes * base['Diff']
# Vendo total de ações na carteira
base['Total Acoes'] = base['Mov_Acoes'].cumsum()
base

,Preco_Ativo,Semana,Delta,Diff,Mov_Acoes,Total Acoes
0,100.0,0.019231,0.519356,0.519356,519.355569,519.355569
1,93.4,0.038462,0.047249,-0.472107,-472.106723,47.248846
2,88.7,0.057692,0.007931,-0.039318,-39.317611,7.931235
3,91.2,0.076923,0.058961,0.051029,51.029296,58.960532
4,84.9,0.096154,0.005687,-0.053273,-53.273474,5.687058
5,107.5,0.115385,0.881679,0.875992,875.991654,881.678712
6,116.3,0.134615,0.985601,0.103922,103.922159,985.600872
7,113.4,0.153846,0.959097,-0.026504,-26.503968,959.096904
8,106.7,0.173077,0.822522,-0.136575,-136.574529,822.522375
9,97.9,0.192308,0.464739,-0.357783,-357.783342,464.739033


In [ ]:
# Mesma ideia visão Put
base = pd.DataFrame(data={'Preco_Ativo':S,'Semana':T})
deltas = []
delta_anterior = 0
diffs_deltas = []
mov_acoes = []

for price,time in zip(S,T):
    # Calculando para uma CALL
    preco_put, delta_put, hedge_put = black_scholes(price, K, time, r, sigma, "put", qtd_opcoes)
    deltas.append(delta_put)

    #Fazendo a diff entre deltas para ajudste do Hedge
    diff_delta = delta_put - delta_anterior
    diffs_deltas.append(diff_delta)

    #Segurando o delta para a próxima diff
    delta_anterior = delta_put

base['Delta'] = deltas
base['Diff'] = diffs_deltas
base["Mov_Acoes"] = qtd_opcoes * base['Diff']
base['Total Acoes'] = base['Mov_Acoes'].cumsum()
base

,Preco_Ativo,Semana,Delta,Diff,Mov_Acoes,Total Acoes
0,100.0,0.019231,-0.480644,-0.480644,-480.644431,-480.644431
1,93.4,0.038462,-0.952751,-0.472107,-472.106723,-952.751154
2,88.7,0.057692,-0.992069,-0.039318,-39.317611,-992.068765
3,91.2,0.076923,-0.941039,0.051029,51.029296,-941.039468
4,84.9,0.096154,-0.994313,-0.053273,-53.273474,-994.312942
5,107.5,0.115385,-0.118321,0.875992,875.991654,-118.321288
6,116.3,0.134615,-0.014399,0.103922,103.922159,-14.399128
7,113.4,0.153846,-0.040903,-0.026504,-26.503968,-40.903096
8,106.7,0.173077,-0.177478,-0.136575,-136.574529,-177.477625
9,97.9,0.192308,-0.535261,-0.357783,-357.783342,-535.260967
